In [ ]:
import pandas as pd
import numpy as np
import sklearn
from sklearn.model_selection import StratifiedShuffleSplit, train_test_split
from sklearn.metrics import roc_curve, auc, accuracy_score, confusion_matrix, classification_report
from keras.callbacks import EarlyStopping
import matplotlib.pylab as plt
import os
from keras.models import Sequential, Model
from keras.layers import Input, Dense, Dropout, Flatten, ConvLSTM2D, TimeDistributed, Bidirectional, LSTM
from keras.utils import Sequence
from tensorflow.keras.callbacks import Callback
from keras.callbacks import ModelCheckpoint, EarlyStopping
import matplotlib.pyplot as plt
import tensorflow as tf
import tempfile



class DataGen(Sequence):
    """ A sequence of data for training/test/validation, loaded from memory
    batch by batch. Extends the tensorflow.keras.utils.Sequence: https://www.tensorflow.org/api_docs/python/tf/keras/utils/Sequence

    Attributes
    ----------
    base_path : str
                path to the folder including the samples.
    filenames : list<str>
                list of sample filenames.
    labels : list<str>
             list of sample labels.
    batch_size : int
                 batch size to load samples

    """

    def __init__(self, base_path, filenames, labels, batch_size, Preprocess_input):
        self.base_path = base_path
        self.filenames = filenames
        self.labels = labels
        self.batch_size = batch_size
        self.Preprocess_input = Preprocess_input

    def __len__(self):
        return (np.ceil(len(self.filenames) / float(self.batch_size))).astype(int)

    def __getitem__(self, idx):
        batch_x = self.filenames[idx * self.batch_size: (idx + 1) * self.batch_size]
        batch_y = self.labels[idx * self.batch_size: (idx + 1) * self.batch_size]

        return np.array([self.Preprocess_input(np.load(os.path.join(self.base_path, file_name))) for file_name in batch_x]), np.array(batch_y)

def GetPretrainedModel(ModelConstructor, input_shape=(224, 224, 3), print_summary=True):
    """ Builds the MobileNet V3 Small 2D with the Imagenet weights, freezing all layers except layers_to_finetune

    Parameters
    ----------
    ModelConstructor : Callable[[bool], [str], [tuple], Sequential]
                       Function that download the pretrained model, i.e. one of the Keras applications:
                       https://keras.io/api/applications/
                       The arguments are include_top, weights, and input_shape.
    input_shape : tuple
                  The input shape for the pretrained model.
    print_summary : bool
                    If True prints the model summary.

    Returns
    -------
    model : Sequential
          The instantiated model.
    """

    model = ModelConstructor(include_top=False, weights="imagenet", input_shape=input_shape)

    for layer in model.layers:
        layer.trainable = False

    return model

def getLSTMModel(getConvModel, ModelConstructor, pretrained_input_shape=(224, 224, 3), verbose=True):
    """Creates the BiLSTM + fully connected layers end-to-end model object
    with the sequential API: https://keras.io/models/sequential/

    Parameters
    ----------
    getConvModel : Callable[Callable[[bool], [str], [tuple], Sequential], [tuple], [bool], Sequential]
                Function that instantiates the pretrained Convolutional model
                to be applied in a time distributed fashion.
    ModelConstructor : Callable[[bool], [str], [tuple], Sequential]
                       Function that download the pretrained model, i.e. one of the Keras applications:
                       https://keras.io/api/applications/
                       The arguments are include_top, weights, and input_shape.
    input_shape : tuple
                  The input shape for the pretrained model.
    verbose : bool
              if True prints the model summary (default True)

    Returns
    -------
    model : Sequential
            The instantiated model
    """
    model = Sequential()
    model.add(TimeDistributed(getConvModel(ModelConstructor, pretrained_input_shape, verbose), input_shape=(16, 224, 224, 3)))

    model.add(TimeDistributed(Flatten()))
    model.add(Bidirectional(LSTM(units=128, return_sequences=False)))
    #model.add(LSTM(units=128, return_sequences=False))

    model.add(Dropout(0.5))
    model.add(Dense(128, activation='relu'))

    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))
    if verbose:
        model.summary()
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

    return model

def getConvLSTMModel(getConvModel, ModelConstructor, pretrained_input_shape=(224, 224, 3), verbose=True):
    """Creates the BiLSTM + fully connected layers end-to-end model object
    with the sequential API: https://keras.io/models/sequential/

    Parameters
    ----------
    getConvModel : Callable[Callable[[bool], [str], [tuple], Sequential], [tuple], [bool], Sequential]
                Function that instantiates the pretrained Convolutional model
                to be applied in a time distributed fashion.
    ModelConstructor : Callable[[bool], [str], [tuple], Sequential]
                       Function that download the pretrained model, i.e. one of the Keras applications:
                       https://keras.io/api/applications/
                       The arguments are include_top, weights, and input_shape.
    input_shape : tuple
                  The input shape for the pretrained model.
    verbose : bool
              if True prints the model summary (default True)

    Returns
    -------
    model : Sequential
            The instantiated model
    """
    model = Sequential()
    model.add(TimeDistributed(getConvModel(ModelConstructor, pretrained_input_shape, verbose), input_shape=(16, 224, 224, 3)))

    model.add(ConvLSTM2D(filters=64, kernel_size=(3, 3)))

    model.add(Flatten())
    model.add(Dropout(0.5))
    model.add(Dense(256, activation='relu'))

    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))
    if verbose:
        model.summary()
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

    return model


def runEndToEndExperiment(getLSTMModel, getConvModel, ModelConstructor, pretrained_input_shape, Preprocess_input, batchSize, datasetBasePath, npyBasePath, featuresPath, samplesMMapName, lablesMMapName, endToEndModelName, rState, savePath):
        return  getLSTMModel(getConvModel, ModelConstructor, pretrained_input_shape)



# Definisci il callback personalizzato
class InferenceTimeCallback(Callback):
    def on_predict_batch_begin(self, batch, logs=None):
        self.start_time = time.time()

    def on_predict_batch_end(self, batch, logs=None):
        self.end_time = time.time()
        self.inference_time = self.end_time - self.start_time
        print(f"Inference time for batch {batch}: {self.inference_time} seconds")


In [ ]:
# Mounts Google Drive

from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

In [ ]:
!pip install moviepy
!pip install codecarbon

In [ ]:

import cv2
import numpy as np
import os

def ensure_dir(file_path):
    if not os.path.exists(file_path):
        os.makedirs(file_path)

def compress_jpeg(image, quality):
    encode_param = [int(cv2.IMWRITE_JPEG_QUALITY), quality]
    _, encimg = cv2.imencode('.jpg', image, encode_param)
    decimg = cv2.imdecode(encimg, 1)
    return decimg

def reduce_resolution(image, scale_percent):
    width = int(image.shape[1] * scale_percent / 100)
    height = int(image.shape[0] * scale_percent / 100)
    dim = (width, height)
    resized = cv2.resize(image, dim, interpolation=cv2.INTER_AREA)
    return cv2.resize(resized, (image.shape[1], image.shape[0]), interpolation=cv2.INTER_AREA)

def add_noise(image, mean=0, var=10):
    sigma = var ** 0.5
    gauss = np.random.normal(mean, sigma, image.shape).astype('uint8')
    noisy_image = cv2.add(image, gauss)
    return noisy_image

def blur_image(image, kernel_size=5):
    return cv2.GaussianBlur(image, (kernel_size, kernel_size), 0)

def distort_color(image, alpha=1.5, beta=20):
    return cv2.convertScaleAbs(image, alpha=alpha, beta=beta)

def process_and_save_video(src_path, dst_path, augmentations):
    cap = cv2.VideoCapture(src_path)
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = None
    if cap.isOpened():
        frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
        fps = cap.get(cv2.CAP_PROP_FPS)
        ensure_dir(os.path.dirname(dst_path))
        out = cv2.VideoWriter(dst_path, fourcc, fps, (frame_width, frame_height))

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        for aug in augmentations:
            frame = aug(frame)

        out.write(frame)

    cap.release()
    out.release()

def process_directory(src_dir, dst_dir, augmentations):
    categories = ['Violence', 'NonViolence']
    cameras = ['cam1', 'cam2']

    for category in categories:
        for camera in cameras:
            src_path = os.path.join(src_dir, category, camera)
            dst_path = os.path.join(dst_dir, category, camera)
            ensure_dir(dst_path)

            for filename in os.listdir(src_path):
                if filename.endswith(".mp4"):
                    src_video_path = os.path.join(src_path, filename)
                    dst_video_path = os.path.join(dst_path, filename)
                    process_and_save_video(src_video_path, dst_video_path, augmentations)


source_folder = '/content/gdrive/MyDrive/Dataset/Data-Augmented'
destination_folder = '/content/gdrive/MyDrive/Dataset/Data_Augmented'

augmentations = [
    #lambda img: compress_jpeg(img, quality=70),
    lambda img: reduce_resolution(img, scale_percent=30),
    #lambda img: add_noise(img, var=20),
    lambda img: blur_image(img, kernel_size=25),
    #lambda img: distort_color(img, alpha=1.5, beta=30)
]


process_directory(source_folder, destination_folder, augmentations)


### *Pruning al 50% sul modello ConvLSTM*



*   Peso modello: 12 MB
*   Accuratezza modello: 94.5%
*   Auc: 98%
*   Tempo di inferenza medio per batch: 0.38 s circa
*   Numero di parametri: 9538186
*   Consumo Energetico medio per 102 secondi di video: 0.0023  kW/h



In [ ]:


from tensorflow.keras.applications import MobileNetV3Small
from tensorflow.keras.applications.mobilenet_v3 import preprocess_input as mobilenet_v3_preprocess_input
from tensorflow.keras.preprocessing import image
import matplotlib.pyplot as plt


MobileNetV3Small_ConvLSTM = runEndToEndExperiment(
    getConvLSTMModel,
    GetPretrainedModel,
    MobileNetV3Small,
    (224, 224, 3),
    mobilenet_v3_preprocess_input,
    8,
    '/content/gdrive/My Drive/Dataset/AirtLab-Dataset',
    '/airtlabDataset',
    'features',
    'filenames.npy',
    'labels.npy',
    'MobileNetV3Small + ConvLSTM',
    42,
    '/content/gdrive/ My Drive/MobileNetV3Small_ConvLSTM'
)


MobileNetV3Small_ConvLSTM.load_weights('/content/gdrive/MyDrive/Modelli/MobileNet_V3_Small/ConvLSTM/final_model_fold_5.h5')


In [ ]:
pip install tensorflow-model-optimization

In [ ]:
import tensorflow as tf
from tensorflow import keras
import tensorflow_model_optimization as tfmot
import cv2


def preprocess_video(video_path, input_shape, batch_size):
    cap = cv2.VideoCapture(video_path)
    frames = []
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        frame = cv2.resize(frame, (input_shape[1], input_shape[0]))
        frame = frame.astype('float32')
        frames.append(frame)

    cap.release()
    frames = np.array(frames)


    if len(frames) >= batch_size:
        num_batches = len(frames) // batch_size
        frames = frames[:num_batches * batch_size]
        batches = np.split(frames, num_batches)
    else:
        batches = []

    return batches

def load_videos_from_folder(folder_path, label, input_shape, batch_size):
    videos = []
    labels = []
    for filename in os.listdir(folder_path):
        if filename.endswith(".mp4") or filename.endswith(".avi"):
            video_path = os.path.join(folder_path, filename)
            video_batches = preprocess_video(video_path, input_shape, batch_size)
            videos.extend(video_batches)
            labels.extend([label] * len(video_batches))
    return videos, labels

def apply_pruning_to_supported_layers(model, pruning_params):
    for layer in model.layers:
        if isinstance(layer, (tf.keras.layers.Dense, tf.keras.layers.Conv2D)):
            layer = tfmot.sparsity.keras.prune_low_magnitude(layer, **pruning_params)
    return model



prune_low_magnitude = tfmot.sparsity.keras.prune_low_magnitude
train_violence_path = "/content/gdrive/MyDrive/Dataset/Data_Augmented/Train/Violence"
train_nonviolence_path = "/content/gdrive/MyDrive/Dataset/Data_Augmented/Train/NonViolence"
val_violence_path = "/content/gdrive/MyDrive/Dataset/Data_Augmented/Validation/Violence"
val_nonviolence_path = "/content/gdrive/MyDrive/Dataset/Data_Augmented/Validation/NonViolence"
input_shape = (224, 224)
sequence_length = 16
batch_size = 16
epochs = 2


violence_train_videos, violence_train_labels = load_videos_from_folder(train_violence_path, 1, input_shape, batch_size)
nonviolence_train_videos, nonviolence_train_labels = load_videos_from_folder(train_nonviolence_path, 0, input_shape, batch_size)
violence_val_videos, violence_val_labels = load_videos_from_folder(val_violence_path, 1, input_shape, batch_size)
nonviolence_val_videos, nonviolence_val_labels = load_videos_from_folder(val_nonviolence_path, 0, input_shape, batch_size)

x_train = np.concatenate((violence_train_videos, nonviolence_train_videos), axis=0)
y_train = np.concatenate((violence_train_labels, nonviolence_train_labels), axis=0)
x_test = np.concatenate((violence_val_videos, nonviolence_val_videos), axis=0)
y_test = np.concatenate((violence_val_labels, nonviolence_val_labels), axis=0)


num_images = len(x_train) + len(x_test)
end_step = np.ceil(num_images).astype(np.int32) * epochs

pruning_params = {
    'pruning_schedule': tfmot.sparsity.keras.PolynomialDecay(
        initial_sparsity=0.0,
        final_sparsity=0.5,
        begin_step=0,
        end_step=end_step
    )
}


model_for_pruning = apply_pruning_to_supported_layers(MobileNetV3Small_ConvLSTM, pruning_params)


model_for_pruning.compile(optimizer='adam',
                          loss='binary_crossentropy',
                          metrics=['accuracy'])

model_for_pruning.summary()


logdir = tempfile.mkdtemp()

callbacks = [
    tfmot.sparsity.keras.UpdatePruningStep(),
    tfmot.sparsity.keras.PruningSummaries(log_dir=logdir)
]


model_for_pruning.fit(x_train, y_train,
                      batch_size=batch_size,
                      epochs=epochs,
                      callbacks=callbacks,
                      validation_data=(x_test, y_test))


pruned_model = tfmot.sparsity.keras.strip_pruning(model_for_pruning)


pruned_model.save('/content/gdrive/MyDrive/Modelli/MobileNet_V3_Small/Pruning/50_perc/pruned_model_ConvLSTM.h5')


converter = tf.lite.TFLiteConverter.from_keras_model(pruned_model)


converter.experimental_new_converter = True
converter._experimental_lower_tensor_list_ops = True


converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS,
                                       tf.lite.OpsSet.SELECT_TF_OPS]

pruned_tflite_model = converter.convert()



with open('/content/gdrive/MyDrive/Modelli/MobileNet_V3_Small/Pruning/50_perc/pruned_model_ConvLSTM.tflite', 'wb') as f:
    f.write(pruned_tflite_model)


In [ ]:


import tensorflow as tf


def tflite_model_summary(interpreter):

    interpreter.allocate_tensors()


    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()
    all_tensor_details = interpreter.get_tensor_details()

    layers = {}

    for tensor_detail in all_tensor_details:
        layer_name = tensor_detail['name'].split('/')[0]
        if layer_name not in layers:
            layers[layer_name] = {
                'name': layer_name,
                'output_shape': [],
                'type': [],
                'param_count': 0
            }
        layers[layer_name]['output_shape'].append(tensor_detail['shape'])
        layers[layer_name]['type'].append(str(tensor_detail['dtype']))


    total_params = 0
    for tensor_detail in all_tensor_details:
        shape = tensor_detail['shape']
        param_count = 1
        for dim in shape:
            param_count *= dim
        total_params += param_count
        layer_name = tensor_detail['name'].split('/')[0]
        layers[layer_name]['param_count'] += param_count


    print("_________________________________________________________________")
    print(" Layer (type)                Output Shape              Param #   ")
    print("=================================================================")
    for layer_name, layer_info in layers.items():
        output_shape_str = ' / '.join([str(shape) for shape in layer_info['output_shape']])
        dtype_str = ' / '.join(layer_info['type'])
        print(f" {layer_name} ({dtype_str})  {output_shape_str}     {layer_info['param_count']}")
    print("=================================================================")
    print(f"Total params: {total_params}")
    print("_______________________________________________________________")



pruned_model_path = '/content/gdrive/MyDrive/Modelli/MobileNet_V3_Small/Pruning/50_perc/pruned_model_ConvLSTM.tflite'
interpreter = tf.lite.Interpreter(model_path=pruned_model_path)


tflite_model_summary(interpreter)


In [ ]:


import cv2
import tensorflow as tf
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc, accuracy_score, precision_score, recall_score
import matplotlib.pyplot as plt
import time

def load_tflite_model(model_path):
    interpreter = tf.lite.Interpreter(model_path=model_path)
    interpreter.allocate_tensors()
    return interpreter

def preprocess_video(video_path, input_shape, batch_size):
    cap = cv2.VideoCapture(video_path)
    frames = []
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        frame = cv2.resize(frame, (input_shape[1], input_shape[0]))
        frame = frame.astype('float32')
        frames.append(frame)

    cap.release()
    frames = np.array(frames)


    if len(frames) >= batch_size:
        num_batches = len(frames) // batch_size
        frames = frames[:num_batches * batch_size]
        batches = np.split(frames, num_batches)
    else:
        batches = []

    return batches


def run_inference(interpreter, input_data):
    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()

    interpreter.set_tensor(input_details[0]['index'], input_data)


    start_time = time.time()
    interpreter.invoke()
    end_time = time.time()
    inference_time = end_time - start_time

    output_data = interpreter.get_tensor(output_details[0]['index'])

    return output_data, inference_time

def calculate_metrics(y_true, y_pred, y_probs):
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    specificity = recall_score(y_true, y_pred, pos_label=0)
    fpr, tpr, thresholds = roc_curve(y_true, y_probs)
    roc_auc = auc(fpr, tpr)

    return accuracy, precision, recall, specificity, roc_auc, fpr, tpr

def plot_roc_curve(fpr, tpr, roc_auc, title='ROC Curve'):
    plt.figure()
    plt.plot(fpr, tpr, color='b', lw=2, label='ROC curve (area = %0.4f)' % roc_auc)
    plt.plot([0, 1], [0, 1], color='r', linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title(title)
    plt.legend(loc="lower right")
    plt.show()

def load_videos_from_folder(folder_path, label, input_shape, batch_size):
    videos = []
    labels = []
    for filename in os.listdir(folder_path):
        if filename.endswith(".mp4") or filename.endswith(".avi"):
            video_path = os.path.join(folder_path, filename)
            video_batches = preprocess_video(video_path, input_shape, batch_size)
            videos.extend(video_batches)
            labels.extend([label] * len(video_batches))
    return videos, labels


from codecarbon import track_emissions
@track_emissions(project_name="MV3_Small_Inference_ConvLSTM_pruned_50")
def inferenceV3_ConvLSTM():
    pruned_model_path = '/content/gdrive/MyDrive/Modelli/MobileNet_V3_Small/Pruning/50_perc/pruned_model_ConvLSTM.tflite'
    violence_path = '/content/gdrive/MyDrive/VideoInferenza/Violence'
    nonviolence_path = '/content/gdrive/MyDrive/VideoInferenza/NonViolence'
    input_shape = (224, 224)
    batch_size = 16

    interpreter = load_tflite_model(pruned_model_path)

    violence_videos, violence_labels = load_videos_from_folder(violence_path, 1, input_shape, batch_size)
    nonviolence_videos, nonviolence_labels = load_videos_from_folder(nonviolence_path, 0, input_shape, batch_size)

    all_videos = violence_videos + nonviolence_videos
    all_labels = violence_labels + nonviolence_labels

    all_videos = np.array([np.expand_dims(video, axis=0) for video in all_videos])
    all_labels = np.array(all_labels)

    total_inference_time = 0
    y_probs = []
    for video in all_videos:
        output_data, inference_time = run_inference(interpreter, video)
        total_inference_time += inference_time
        y_probs.append(output_data.ravel()[0]) # / 255.0)

    y_probs = np.array(y_probs)
    print(f"Probabilità {y_probs}")
    y_pred = np.round(y_probs)



    print("Unique values in all_labels:", np.unique(all_labels))
    print("Unique values in y_pred:", np.unique(y_pred))


    accuracy, precision, recall, specificity, roc_auc, fpr, tpr = calculate_metrics(all_labels, y_pred, y_probs)


    average_inference_time = total_inference_time / len(all_videos)


    print(f'Accuracy: {accuracy:.4f}')
    print(f'Precision: {precision:.4f}')
    print(f'Recall: {recall:.4f}')
    print(f'Specificity: {specificity:.4f}')
    print(f'ROC AUC: {roc_auc:.4f}')
    print(f'Average Inference Time: {average_inference_time:.4f} seconds')


    print('Classification Report:')
    print(classification_report(all_labels, y_pred))

    print('Confusion Matrix:')
    print(confusion_matrix(all_labels, y_pred))


    plot_roc_curve(fpr, tpr, roc_auc, title='ROC Curve for tf_model_mv3_convLStm')

if __name__ == '__main__':
    inferenceV3_ConvLSTM()


### *Pruning al 50% sul modello BiLSTM*


*   Peso modello: 114.5 MB
*   Accuratezza modello: 74%
*   Auc: 97.6%
*   Tempo di inferenza medio per batch: 0.37 s circa
*   Numero di parametri: 64470815
*   Consumo Energetico medio per 102 secondi di video: 0.001998  kW/h

In [ ]:

from tensorflow.keras.applications import MobileNetV3Small
from tensorflow.keras.applications.mobilenet_v3 import preprocess_input as mobilenet_v3_preprocess_input
from tensorflow.keras.preprocessing import image
import matplotlib.pyplot as plt
from tensorflow.keras.callbacks import Callback



MobileNetV3Small_BiLSTM = runEndToEndExperiment(
    getLSTMModel,
    GetPretrainedModel,
    MobileNetV3Small,
    (224, 224, 3),
    mobilenet_v3_preprocess_input,
    8,
    '/content/gdrive/My Drive/Dataset/AirtLab-Dataset',
    '/airtlabDataset',
    'features',
    'filenames.npy',
    'labels.npy',
    'MobileNetV3Small + BiLSTM',
    42,
    '/content/gdrive/ My Drive/MobileNetV3Small_BiLSTM'
)


MobileNetV3Small_BiLSTM.load_weights('/content/gdrive/MyDrive/Modelli/MobileNet_V3_Small/BiLSTM/final_model_fold_5.h5')



In [ ]:
!pip install tensorflow_model_optimization

In [ ]:
import tensorflow as tf
from tensorflow import keras
import tensorflow_model_optimization as tfmot
import cv2


def preprocess_video(video_path, input_shape, batch_size):
    cap = cv2.VideoCapture(video_path)
    frames = []
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        frame = cv2.resize(frame, (input_shape[1], input_shape[0]))
        frame = frame.astype('float32')
        frames.append(frame)

    cap.release()
    frames = np.array(frames)


    if len(frames) >= batch_size:
        num_batches = len(frames) // batch_size
        frames = frames[:num_batches * batch_size]
        batches = np.split(frames, num_batches)
    else:
        batches = []

    return batches

def load_videos_from_folder(folder_path, label, input_shape, batch_size):
    videos = []
    labels = []
    for filename in os.listdir(folder_path):
        if filename.endswith(".mp4") or filename.endswith(".avi"):
            video_path = os.path.join(folder_path, filename)
            video_batches = preprocess_video(video_path, input_shape, batch_size)
            videos.extend(video_batches)
            labels.extend([label] * len(video_batches))
    return videos, labels


def apply_pruning_to_supported_layers(model, pruning_params):
    for layer in model.layers:
        if isinstance(layer, (tf.keras.layers.Dense, tf.keras.layers.Conv2D)):
            layer = tfmot.sparsity.keras.prune_low_magnitude(layer, **pruning_params)
    return model


prune_low_magnitude = tfmot.sparsity.keras.prune_low_magnitude
train_violence_path = "/content/gdrive/MyDrive/Dataset/Data_Augmented/Train/Violence"
train_nonviolence_path = "/content/gdrive/MyDrive/Dataset/Data_Augmented/Train/NonViolence"
val_violence_path = "/content/gdrive/MyDrive/Dataset/Data_Augmented/Validation/Violence"
val_nonviolence_path = "/content/gdrive/MyDrive/Dataset/Data_Augmented/Validation/NonViolence"
input_shape = (224, 224)
sequence_length = 16
batch_size = 16
epochs = 2


violence_train_videos, violence_train_labels = load_videos_from_folder(train_violence_path, 1, input_shape, batch_size)
nonviolence_train_videos, nonviolence_train_labels = load_videos_from_folder(train_nonviolence_path, 0, input_shape, batch_size)
violence_val_videos, violence_val_labels = load_videos_from_folder(val_violence_path, 1, input_shape, batch_size)
nonviolence_val_videos, nonviolence_val_labels = load_videos_from_folder(val_nonviolence_path, 0, input_shape, batch_size)

x_train = np.concatenate((violence_train_videos, nonviolence_train_videos), axis=0)
y_train = np.concatenate((violence_train_labels, nonviolence_train_labels), axis=0)
x_test = np.concatenate((violence_val_videos, nonviolence_val_videos), axis=0)
y_test = np.concatenate((violence_val_labels, nonviolence_val_labels), axis=0)


num_images = len(x_train) + len(x_test)
end_step = np.ceil(num_images).astype(np.int32) * epochs

pruning_params = {
    'pruning_schedule': tfmot.sparsity.keras.PolynomialDecay(
        initial_sparsity=0.0,
        final_sparsity=0.5,
        begin_step=0,
        end_step=end_step
    )
}


model_for_pruning_biLSTM = apply_pruning_to_supported_layers(MobileNetV3Small_BiLSTM, pruning_params)


model_for_pruning_biLSTM.compile(optimizer='adam',
                          loss='binary_crossentropy',
                          metrics=['accuracy'])

model_for_pruning_biLSTM.summary()


logdir = tempfile.mkdtemp()

callbacks = [
    tfmot.sparsity.keras.UpdatePruningStep(),
    tfmot.sparsity.keras.PruningSummaries(log_dir=logdir)
]


model_for_pruning_biLSTM.fit(x_train, y_train,
                      batch_size=batch_size,
                      epochs=epochs,
                      callbacks=callbacks,
                      validation_data=(x_test, y_test))


pruned_model_biLSTM = tfmot.sparsity.keras.strip_pruning(model_for_pruning_biLSTM)


pruned_model_biLSTM.save('/content/gdrive/MyDrive/Modelli/MobileNet_V3_Small/Pruning/50_perc/pruned_model_BiLSTM.h5')


converter = tf.lite.TFLiteConverter.from_keras_model(pruned_model_biLSTM)


converter.experimental_new_converter = True
converter._experimental_lower_tensor_list_ops = True


converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS,
                                       tf.lite.OpsSet.SELECT_TF_OPS]

pruned_tflite_model_biLSTM = converter.convert()



with open('/content/gdrive/MyDrive/Modelli/MobileNet_V3_Small/Pruning/50_perc/pruned_model_BiLSTM.tflite', 'wb') as f:
    f.write(pruned_tflite_model_biLSTM)


In [ ]:


import tensorflow as tf


def tflite_model_summary(interpreter):

    interpreter.allocate_tensors()


    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()
    all_tensor_details = interpreter.get_tensor_details()

    layers = {}

    for tensor_detail in all_tensor_details:
        layer_name = tensor_detail['name'].split('/')[0]
        if layer_name not in layers:
            layers[layer_name] = {
                'name': layer_name,
                'output_shape': [],
                'type': [],
                'param_count': 0
            }
        layers[layer_name]['output_shape'].append(tensor_detail['shape'])
        layers[layer_name]['type'].append(str(tensor_detail['dtype']))


    total_params = 0
    for tensor_detail in all_tensor_details:
        shape = tensor_detail['shape']
        param_count = 1
        for dim in shape:
            param_count *= dim
        total_params += param_count
        layer_name = tensor_detail['name'].split('/')[0]
        layers[layer_name]['param_count'] += param_count


    print("_________________________________________________________________")
    print(" Layer (type)                Output Shape              Param #   ")
    print("=================================================================")
    for layer_name, layer_info in layers.items():
        output_shape_str = ' / '.join([str(shape) for shape in layer_info['output_shape']])
        dtype_str = ' / '.join(layer_info['type'])
        print(f" {layer_name} ({dtype_str})  {output_shape_str}     {layer_info['param_count']}")
    print("=================================================================")
    print(f"Total params: {total_params}")
    print("_______________________________________________________________")



pruned_model_biLSTM_path = "/content/gdrive/MyDrive/Modelli/MobileNet_V3_Small/Pruning/50_perc/pruned_model_BiLSTM.tflite"
interpreter = tf.lite.Interpreter(model_path=pruned_model_biLSTM_path)


tflite_model_summary(interpreter)


In [ ]:
!pip install tensorflow-model-optimization

In [ ]:


import cv2
import tensorflow as tf
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc, accuracy_score, precision_score, recall_score
import matplotlib.pyplot as plt
import time

def load_tflite_model(model_path):
    interpreter = tf.lite.Interpreter(model_path=model_path)
    interpreter.allocate_tensors()
    return interpreter

def preprocess_video(video_path, input_shape, batch_size):
    cap = cv2.VideoCapture(video_path)
    frames = []
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        frame = cv2.resize(frame, (input_shape[1], input_shape[0]))
        frame = frame.astype('float32')
        frames.append(frame)

    cap.release()
    frames = np.array(frames)


    if len(frames) >= batch_size:
        num_batches = len(frames) // batch_size
        frames = frames[:num_batches * batch_size]
        batches = np.split(frames, num_batches)
    else:
        batches = []

    return batches


def run_inference(interpreter, input_data):
    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()

    interpreter.set_tensor(input_details[0]['index'], input_data)


    start_time = time.time()
    interpreter.invoke()
    end_time = time.time()
    inference_time = end_time - start_time

    output_data = interpreter.get_tensor(output_details[0]['index'])

    return output_data, inference_time

def calculate_metrics(y_true, y_pred, y_probs):
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    specificity = recall_score(y_true, y_pred, pos_label=0)
    fpr, tpr, thresholds = roc_curve(y_true, y_probs)
    roc_auc = auc(fpr, tpr)

    return accuracy, precision, recall, specificity, roc_auc, fpr, tpr

def plot_roc_curve(fpr, tpr, roc_auc, title='ROC Curve'):
    plt.figure()
    plt.plot(fpr, tpr, color='b', lw=2, label='ROC curve (area = %0.4f)' % roc_auc)
    plt.plot([0, 1], [0, 1], color='r', linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title(title)
    plt.legend(loc="lower right")
    plt.show()

def load_videos_from_folder(folder_path, label, input_shape, batch_size):
    videos = []
    labels = []
    for filename in os.listdir(folder_path):
        if filename.endswith(".mp4") or filename.endswith(".avi"):
            video_path = os.path.join(folder_path, filename)
            video_batches = preprocess_video(video_path, input_shape, batch_size)
            videos.extend(video_batches)
            labels.extend([label] * len(video_batches))
    return videos, labels


from codecarbon import track_emissions
@track_emissions(project_name="MV3_Small_Inference_BiLSTM_pruned_50")
def inferenceV3_BiLSTM():
    pruned_model_biLSTM_path = '/content/gdrive/MyDrive/Modelli/MobileNet_V3_Small/Pruning/50_perc/pruned_model_BiLSTM.tflite'
    violence_path = '/content/gdrive/MyDrive/VideoInferenza/Violence'
    nonviolence_path = '/content/gdrive/MyDrive/VideoInferenza/NonViolence'
    input_shape = (224, 224)
    batch_size = 16

    interpreter = load_tflite_model(pruned_model_biLSTM_path)

    violence_videos, violence_labels = load_videos_from_folder(violence_path, 1, input_shape, batch_size)
    nonviolence_videos, nonviolence_labels = load_videos_from_folder(nonviolence_path, 0, input_shape, batch_size)

    all_videos = violence_videos + nonviolence_videos
    all_labels = violence_labels + nonviolence_labels

    all_videos = np.array([np.expand_dims(video, axis=0) for video in all_videos])
    all_labels = np.array(all_labels)

    total_inference_time = 0
    y_probs = []
    for video in all_videos:
        output_data, inference_time = run_inference(interpreter, video)
        total_inference_time += inference_time
        y_probs.append(output_data.ravel()[0]) # / 255.0)

    y_probs = np.array(y_probs)
    print(f"Probabilità {y_probs}")
    y_pred = np.round(y_probs)



    print("Unique values in all_labels:", np.unique(all_labels))
    print("Unique values in y_pred:", np.unique(y_pred))


    accuracy, precision, recall, specificity, roc_auc, fpr, tpr = calculate_metrics(all_labels, y_pred, y_probs)


    average_inference_time = total_inference_time / len(all_videos)


    print(f'Accuracy: {accuracy:.4f}')
    print(f'Precision: {precision:.4f}')
    print(f'Recall: {recall:.4f}')
    print(f'Specificity: {specificity:.4f}')
    print(f'ROC AUC: {roc_auc:.4f}')
    print(f'Average Inference Time: {average_inference_time:.4f} seconds')


    print('Classification Report:')
    print(classification_report(all_labels, y_pred))

    print('Confusion Matrix:')
    print(confusion_matrix(all_labels, y_pred))


    plot_roc_curve(fpr, tpr, roc_auc, title='ROC Curve for tf_model_mv3_BiLStm_prunato_50')

if __name__ == '__main__':
    inferenceV3_BiLSTM()
